# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [1]:
# 程式區塊 A：將需要的都import進來，設定好要使用的data路徑。
import os
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import math
import warnings
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model  import LogisticRegression
warnings.filterwarnings('ignore')
%matplotlib inline

# 設定【data的資料夾路徑】，命名為【data_folder】
data_folder = 'C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data'

In [2]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_train = os.path.join(data_folder, 'titanic_train.csv')
t002_train = pd.read_csv(t001_train)
print('Path of read in data: %s' %t001_train)
print(t002_train.shape)
t002_train.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_train.csv
(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_test  = os.path.join(data_folder,  'titanic_test.csv')
t002_test  = pd.read_csv(t001_test)
print('Path of read in data: %s' %t001_test)
print(t002_test.shape)
t002_test.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_test.csv
(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# 程式區塊 B-1：做t003，train取【Survived】、test取【PassengerId】
t003_train = t002_train['Survived']
t003_test  = t002_test['PassengerId']
# 程式區塊 B-2：做t004，train捨棄【PassengerId,Survived】、test捨棄【PassengerId】。
t004_train = t002_train.drop(['PassengerId', 'Survived'] , axis=1)
t004_test  = t002_test.drop(['PassengerId'] , axis=1)
print(t004_train.shape)
print(t004_test.shape)

(891, 10)
(418, 10)


In [5]:
# 程式區塊 B-3：設計t005，把train和test合併，UNION ALL的概念。
t005 = pd.concat([t004_train,t004_test])
print(t005.shape)

(1309, 10)


In [6]:
# 秀出資料欄位的類型與數量
t006 = t005.dtypes.reset_index()
t006

,index,0
0,Pclass,int64
1,Name,object
2,Sex,object
3,Age,float64
4,SibSp,int64
5,Parch,int64
6,Ticket,object
7,Fare,float64
8,Cabin,object
9,Embarked,object


以下是教材答案的寫法，有夠混淆的！邏輯次序錯亂，搞不懂自己在做什麼！  
欄位名稱和函式特定用法相同，這要怎麼讓學員知道自己寫這條式子是為了做什麼？  

秀出資料欄位的類型與數量  
   dtype_df = df.dtypes.reset_index()  
1. dtype_df.columns = ["Count", "Column Type"]  
2. dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()  
   dtype_df  

一定是先2.才1.！  
1.的目的在於rename，重命名欄位，但先做這會有個問題啊！資料內容都沒變，但欄位名稱變了，如果我在做2這件事之前，先做了1，並把1印出，  
那這資料就是很有問題的，因為欄位已經變成了【"Count", "Column Type"】，但資料內容卻還是原本的。  
所有的資料處理，一定都是處理完了資料，才來對新的資料作欄位命名，所以當然要先做2.，才能做1.  
2.的目的在於想了解不同的資料欄位類型(int,float,object...)在整份資料裡各有幾個欄位，先把這個做出來，才命名。  

In [7]:
# 先把想知道的事情做出來
t007 = t006.groupby(0).aggregate('count').reset_index()
t007

,0,index
0,int64,3
1,float64,2
2,object,5


In [8]:
# 再針對做完的資料欄位重新命名
t007.columns = ["Column Type","Count" ]
t007

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


除了資料整理的邏輯是上述這樣的次序才正確之外  
當我【做完資料處理、重命名欄位之前】時印出，我並不會對於【0,index】這樣的欄位名稱有困惑，但反過來會。  
當我【做完重命名欄位，資料處理之前】時印出，我會對於欄位是【"Column Type","Count"】的這份資料感到困惑。  

更重要的是，要讓學員知道，它的.aggregate('count')，的這個'count'，並不是欄位名稱！  
而是【.aggregate('count')】的特定用法，裡面總之就是寫上這五個字母：【'count'】。而與欄位名稱無關！  
按照原教材，學員先重命名欄位，萬一現在學員試圖不命名成count，而命名成cnt，接著誤以為資料整理時要寫成【.aggregate('cnt')】  
那他是不是會無法發現自己到底錯在哪？先做.aggregate('count')就不會有這問題嘛！

In [13]:
# 確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
others = []
for a, b in zip(t005.dtypes, t005.columns):
    if   a == 'float64':
        float_features.append(b)
    elif a == 'int64':
        int_features.append(b)
    elif a == 'object':
        object_features.append(b)    
    else:
        others.append(b)  
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}\n')
print(f'{len(others)} Others : {others}')
# 原教材答案中把object放在else...把其中一種重要的分類放在else是很不好的資料處理習慣。
# else永遠都要放【其他】、【剩下的】、【沒辦法被分類的】、【不想分類的】。

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

0 Others : []


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [14]:
# 例 : 整數 (int) 特徵取平均 (mean)
t005[int_features].mean()

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

In [15]:
# 整數 (int) 特徵取最大值 (max)
t005[int_features].max()

Pclass    3
SibSp     8
Parch     9
dtype: int64

In [16]:
# 整數 (int) 特徵取相異值 (nunique)
t005[int_features].nunique()

Pclass    3
SibSp     7
Parch     8
dtype: int64

In [17]:
# 小數 (float) 特徵取平均 (mean)
t005[float_features].mean()

Age     29.881138
Fare    33.295479
dtype: float64

In [18]:
# 小數 (float) 特徵取最大值 (max)
t005[float_features].max()

Age      80.0000
Fare    512.3292
dtype: float64

In [19]:
# 小數 (float) 特徵取相異值 (nunique)
t005[float_features].nunique()

Age      98
Fare    281
dtype: int64

In [20]:
# 類別 (object) 特徵取平均 (mean)
t005[object_features].mean()

Series([], dtype: float64)

In [22]:
# 類別 (object) 特徵取最大值 (max)
t005[object_features].max()

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

In [23]:
# 類別 (object) 特徵取相異值 (nunique)
t005[object_features].nunique()

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64

In [ ]:
### Day18教材方向和目標
1. 三種類型(int/float/object)的欄位分別進行(平均mean/最大值Max/相異值nunique)的狀況
2. for a,b in zip(case_a,case_b)的用法
3. data.groupby(0).aggregate('count')的用法

### Day18忽略部分
1. 對於t006,t007是為了觀察欄位類別類型的這個作法，不以為然，不用改欄位名稱就能看得出來，不用做t007這步。  
2. 做出三個list需要這麼麻煩嘛？需要用到【for in zip】？不能單純拿t005挑就好？SQL裡就只是where而已。
   還是Python建這種東西都是用這種形式？持續觀察。

### Day18其他補充
無